# Example: using a different force field

Here we show how to use MDPOW to calculate partition coefficients using an force field that isn't included in the package. To follow along, install `jupyter` in your `mdpow` environment.

To implement a new force field, you will need:

* `ITP` files for the molecule, the solvents, ions and also the general atom type definitions file (usually just named after the force field itself).
* `MDP` files for the energy minimisation, initial relaxation, NPT ensemble run and free energy calculation.
* Structure files (`.gro` or `.pdb`) for the solute and non-aqueous solvent. If you are using a type of water that does not come bundled with GROMACS, like in this example, you will also need to create an equilibrated box of pure water.

The first thing we'll do is to download the files we need for Martini 3.0.

In [1]:
from pathlib import Path
from typing import Optional

import gromacs
import requests as req


HERE = Path(".")
MARTINI_FF = HERE / "martini.ff"
MARTINI_FF.mkdir(exist_ok=True)

MARTINI_ITP = MARTINI_FF / "forcefield.itp"
MARTINI_IONS = MARTINI_FF / "martini_v3.0.0_ions_v1.itp"
MARTINI_SMALL_MOLS = MARTINI_FF / "martini_v3.0.0_small_molecules_v1.itp"
MARTINI_SOLVENTS = MARTINI_FF / "martini_v3.0.0_solvents_v1.itp"
BENZENE_ITP = MARTINI_SMALL_MOLS

MARTINI_WATER = HERE / "water.gro"
MARTINI_OCTANOL = HERE / "octanol.gro"
MARTINI_BENZENE = HERE / "benzene.pdb"

EM_FILE = HERE / "em.mdp"
EQ_FILE = HERE / "eq.mdp"
RUN_FILE = HERE / "run.mdp"


def download_file(
    url: str, out: Optional[Path] = None, chunk_size: int = 128, overwrite: bool = False
):
    """Utility function to download files."""
    if out is None:
        out = HERE / Path(url).name

    if out.exists() and not overwrite:
        return

    r = req.get(url, stream=True)
    r.raise_for_status()

    with out.open("wb") as f:
        for chunk in r.iter_content(chunk_size=chunk_size):
            f.write(chunk)


DOWNLOADS = {
    MARTINI_ITP: "https://raw.githubusercontent.com/marrink-lab/martini-forcefields/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0.itp",
    MARTINI_IONS: "https://raw.githubusercontent.com/marrink-lab/martini-forcefields/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0_ions_v1.itp",
    MARTINI_SMALL_MOLS: "https://raw.githubusercontent.com/marrink-lab/martini-forcefields/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0_small_molecules_v1.itp",
    MARTINI_SOLVENTS: "https://raw.githubusercontent.com/marrink-lab/martini-forcefields/main/martini_forcefields/regular/v3.0.0/gmx_files/martini_v3.0.0_solvents_v1.itp",
}

for fname, url in DOWNLOADS.items():
    download_file(url, fname)

This should have downloaded several files to your workspace.

We also need to make a `watermodels.dat` file in the `martini.ff` subdirectory.


In [2]:
WATERMODEL_DAT = MARTINI_FF / "watermodels.dat"

WATERMODEL_DAT.write_text("martini-water\tMARTINI-WATER\tMartini default water model.")

56

Next, we set up the files for the Martini 3.0 forcefield.

In [3]:
from mdpow.forcefields import Forcefield, GromacsSolventModel

MARTINI = Forcefield(
    "Martini",
    solvent_models={
        "octanol": GromacsSolventModel(
            identifier="octanol",
            itp=MARTINI_SOLVENTS.absolute(),
            coordinates=str(MARTINI_OCTANOL.absolute()),
            forcefield="Martini",
        ),
    },
    forcefield_dir=MARTINI_FF.absolute(),
    ions_itp=MARTINI_IONS.absolute(),
    default_water_itp=MARTINI_SOLVENTS.absolute(),
    default_water_model="martini-water",
    water_models={
        "martini-water": GromacsSolventModel(
            identifier="martini-water",
            itp=MARTINI_SOLVENTS.absolute(),
            coordinates=str(MARTINI_WATER.absolute()),
            forcefield="Martini",
        ),
    },
)

mdpow       : INFO     MDPOW 0+untagged.616.gf004745.dirty starting.
mdpow       : INFO     Copyright (c) 2010-2023 Shujie Fan, Ian Kenney, Alia Lescoulie, Cade Duckworth, Bogdan Iorga, and Oliver Beckstein
mdpow       : INFO     Released under the GNU Public Licence, version 3.
mdpow       : INFO     For bug reports and help: https://github.com/Becksteinlab/MDPOW/issues
mdpow.config: INFO     Using the bundled force fields from GMXLIB='/home/awsm/MDPOW/mdpow/top'.
mdpow.config: INFO     If required, override this behaviour by setting the environment variable GMXLIB yourself.


## Aqueous simulation

We now solvate a benzene molecule in a water box. The first step of this is to set up a `WaterSimulation`, then perform some energy minimisation and create the files needed for relaxation.

In [4]:
from mdpow.equil import WaterSimulation

sim = WaterSimulation(
    molecule="BENZ",
    forcefield=MARTINI,
    mdp={
        "energy_minimize": str(EM_FILE.absolute()),
        "MD_relaxed": str(EQ_FILE.absolute()),
        "MD_NPT": str(EQ_FILE.absolute()),
        "MD_restrained": str(RUN_FILE.absolute()),
    },
    distance=3.0,
)
sim.topology(str(BENZENE_ITP))
sim.solvate(struct=MARTINI_BENZENE, maxwarn=1)
sim.energy_minimize(maxwarn=1)
sim.MD_relaxed(runtime=1e3, dt=0.01)

mdpow.equil : INFO     [/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top] Created topology 'system.top' that includes 'martini_v3.0.0_small_molecules_v1.itp'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation] Solvating with water '/home/awsm/MDPOW/doc/examples/martini/water.gro'...
                     :-) GROMACS - gmx editconf, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation
Command line:
  gmx editconf -f /home/awsm/MDPOW/doc/examples/martini/benzene.pdb -o boxed.gro -bt dodecahedron -d 3.0


Back Off! I just backed up boxed.gro to ./#boxed.gro.2#

GROMACS reminds you: "Stay Cool, This is a Robbery" (Pulp Fiction)

                     :-) GROMACS - gmx solvate, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3 atoms
Volume: 4050 nm^3, corresponds to roughly 1822500 electrons
No velocities found
    system size :  0.210  0.160  0.248 (nm)
    diameter    :  0.270               (nm)
    center      :  2.987  0.606  2.315 (nm)
    box vectors : 15.000 15.000 18.000 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :4050.00               (nm^3)
    shift       :  1.716  4.097 -0.098 (nm)
new center      :  4.703  4.703  2.217 (nm)
new box vectors :  6.270  6.270  6.270 (nm)
new box angles  :  60.00  60.00  90.00 (degrees)
new box volume  : 174.32               (nm^3)

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functiona

gromacs.cbook: INFO     system total charge qtot = 0
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation] After solvation: total charge qtot = 0 = 0
gromacs.cbook: INFO     system total charge qtot = 0
gromacs.setup: INFO     Building the main index file 'main.ndx'...
                     :-) GROMACS - gmx make_ndx, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation
Command line:
  gmx make_ndx -f ionized.tpr -o main.ndx


Reading structure file
Reading file ionized.tpr, VERSION 2023.2 (single precision)
Reading file ionized.tpr, VERSION 2023.2 (single precision)

Back Off! I just backed up main.ndx to ./#main.ndx.4#

GROMACS reminds you: "A professor is one who talks in someone else's sleep." (W.H. Auden)

                     :-) GROMACS - gmx make_ndx, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gr

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 4: '__main__'
Select group for output
Selected 0: 'System'


atom name 638 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 639 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 640 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 641 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 642 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 643 i

...

atom name 638 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 639 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 640 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 641 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 642 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 643 i

Setting the LD random seed to -1143474855

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'W'

atom name 638 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 639 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 640 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom name 641 in /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top and /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/solvated.gro does not match (W - W 1)

atom 


Back Off! I just backed up em.log to ./#em.log.2#
Reading file em.tpr, VERSION 2023.2 (single precision)
1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 4 OpenMP threads 


Back Off! I just backed up em.trr to ./#em.trr.2#

Back Off! I just backed up em.edr to ./#em.edr.2#

Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+01
   Number of steps    =         1000
Step=    0, Dmax= 1.0e-02 nm, Epot=  9.59019e+06 Fmax= 1.69656e+07, atom= 149
Step=    1, Dmax= 1.0e-02 nm, Epot=  6.21902e+06 Fmax= 6.01139e+06, atom= 478
Step=    2, Dmax= 1.2e-02 nm, Epot=  3.58839e+06 Fmax= 2.70774e+06, atom= 17
Step=    3, Dmax= 1.4e-02 nm, Epot=  2.18086e+06 Fmax= 1.08770e+06, atom= 1305
Step=    4, Dmax= 1.7e-02 nm, Epot=  1.25600e+06 Fmax= 5.29430e+05, atom= 1493
Step=    5, Dmax= 2.1e-02 nm, Epot=

Setting the LD random seed to -10618535

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'W'

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.132 nm, buffer size 0.032 nm

Set rlist, assuming 4x4 atom pair-list, to 1.108 nm, buffer size 0.008 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 1 Mb of data


{'struct': '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/MD_relaxed/md.gro',
 'top': '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/system.top',
 'ndx': '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/main.ndx',
 'qscript': ['./local.sh'],
 'mainselection': None,
 'deffnm': 'md',
 'includes': ['/home/awsm/MDPOW/mdpow/top',
  '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top']}

We can execute `mdrun` using `GromacsWrapper`:

In [5]:
r = gromacs.run.MDrunner(
    dirname=sim.dirs["MD_relaxed"],
    deffnm="md",
    c="md.pdb",
    cpi=True,
    v=True,
)
r.run()  # runs mdrun in the python shell

gromacs.run : INFO     gmx mdrun -deffnm md -c md.pdb -cpi -v
                      :-) GROMACS - gmx mdrun, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/MD_relaxed
Command line:
  gmx mdrun -deffnm md -c md.pdb -cpi -v

Reading file md.tpr, VERSION 2023.2 (single precision)
Changing nstlist from 10 to 50, rlist from 1.107 to 1.268

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
Using 1 MPI thread
Using 4 OpenMP threads 

starting mdrun 'Compound BENZ in water'
100000 steps,   1000.0 ps (continuing from step 100000,   1000.0 ps).

Writing final coordinates.

Back Off! I just backed up md.pdb to ./#md.pdb.2#
step 100000, remaining wall clock time:     0 s          
NOTE: 28 % of the run time was spen

0

Now we do the same thing for a longer NPT run, with a longer timestep.

In [6]:
sim.MD(
    runtime=1e4, qscript=["local.sh"], dt=0.02
)

r = gromacs.run.MDrunner(
    dirname=sim.dirs["MD_NPT"], deffnm="md", c="md.pdb", cpi=True, v=True
)
r.run()  # runs mdrun in the python shell

mdpow.equil : INFO     Found starting structure '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/MD_relaxed/md.pdb' (instead of '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/MD_relaxed/md.gro').
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/MD_NPT] Setting up MD...
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/MD_NPT] input mdp  = '/home/awsm/MDPOW/doc/examples/martini/eq.mdp'
gromacs.cbook: INFO     editing mdp = '/home/awsm/MDPOW/doc/examples/martini/eq.mdp': dict_keys(['nsteps', 'dt', 'pp', 't', 'include'])
gromacs.cbook: WARNING  Not substituted in 'md.mdp': ['pp', 't']
gromacs.setup: WARNING  Unprocessed mdp option are interpreted as options for grompp:
{'pp': 'processed.top', 't': '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/MD_relaxed/md.cpt'}
/home/awsm/mambaforge/envs/mdpow/lib/python3.11/site-packages/gromacs/setup.py:635: UsageWarning: Unprocessed mdp option are interpreted a

Setting the LD random seed to -914915651

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'W'

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.175 nm, buffer size 0.075 nm

Set rlist, assuming 4x4 atom pair-list, to 1.133 nm, buffer size 0.033 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

Reading Coordinates, Velocities and Box size from old trajectory

Will read whole trajectory

Using frame at t = 1000 ps

Starting time for run is 0 ps

This run will generate roughly 6 Mb of data


Reading file md.tpr, VERSION 2023.2 (single precision)
Changing nstlist from 10 to 25, rlist from 1.133 to 1.262

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
Using 1 MPI thread
Using 4 OpenMP threads 

starting mdrun 'Compound BENZ in water'
500000 steps,  10000.0 ps (continuing from step 500000,  10000.0 ps).

Writing final coordinates.

Back Off! I just backed up md.pdb to ./#md.pdb.2#
step 500000, remaining wall clock time:     0 s          
NOTE: 17 % of the run time was spent in pair search,
      you might want to increase nstlist (this has no effect on accuracy)

               Core t (s)   Wall t (s)        (%)
       Time:        0.036        0.009      394.9
                 (ns/day)    (hour/ns)
Performance:      189.866        0.126

GROMACS reminds you: "Meet Me At the Coffee Shop" (Red Hot C

0

Finally, we can set up the thermodynamic integration files. The following will create several directories, one for each $\lambda$ value:

In [7]:
import mdpow.fep

gwat = mdpow.fep.Ghyd(simulation=sim, runtime=1e+3, mdp=str(RUN_FILE.absolute()))
gwat.setup(dt=0.02)

# run multiple simulations on cluster

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

****** PyMBAR will use 64-bit JAX! *******
* JAX is currently set to 32-bit bitsize *
* which is its default.                  *
*                                        *
* PyMBAR requires 64-bit mode and WILL   *
* enable JAX's 64-bit mode when called.  *
*                                        *
* This MAY cause problems with other     *
* Uses of JAX in the same code.          *
******************************************

mdpow.fep   : INFO     Found starting structure '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/MD_NPT/md.pdb' (instead of '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/MD

Setting the LD random seed to -184879625

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'W'

Coupling 1 copies of molecule type 'BENZ'

Velocities were taken from a Maxwell distribution at 300 K

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.297 nm, buffer size 0.197 nm

Set rlist, assuming 4x4 atom pair-list, to 1.214 nm, buffer size 0.114 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 2 Mb of data


Number of degrees of freedom in T-Coupling group System is 4902.00

There were 2 NOTEs

There was 1 WARNING

GROMACS reminds you: "I hadn't been aware that there were doors closed to me until I started knocking on them." (Gertrude Elion)

gromacs.qsub: INFO     Setting up queuing system script './local.sh'...
gromacs.cbook: INFO     editing txt = '/home/awsm/mambaforge/envs/mdpow/lib/python3.11/site-packages/gromacs/templates/local.sh' (9 substitutions)
gromacs.cbook: INFO     edited txt = './local.sh'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0250] output mdp = 'md.mdp'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0250] output ndx = '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/main.ndx'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0250] output tpr = 'md.tpr'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0250] output run

Setting the LD random seed to -603981859

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'W'

Coupling 1 copies of molecule type 'BENZ'

Velocities were taken from a Maxwell distribution at 300 K

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.297 nm, buffer size 0.197 nm

Set rlist, assuming 4x4 atom pair-list, to 1.214 nm, buffer size 0.114 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 2 Mb of data


Number of degrees of freedom in T-Coupling group System is 4902.00

There were 2 NOTEs

There was 1 WARNING

GROMACS reminds you: "Cut It Deep and Cut It Wide" (The Walkabouts)

gromacs.qsub: INFO     Setting up queuing system script './local.sh'...
gromacs.cbook: INFO     editing txt = '/home/awsm/mambaforge/envs/mdpow/lib/python3.11/site-packages/gromacs/templates/local.sh' (9 substitutions)
gromacs.cbook: INFO     edited txt = './local.sh'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0500] output mdp = 'md.mdp'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0500] output ndx = '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/main.ndx'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0500] output tpr = 'md.tpr'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0500] output runscripts = ['./local.sh']
gromacs.setup: INFO     [/home/awsm/

...

Number of degrees of freedom in T-Coupling group System is 4902.00

There were 2 NOTEs

There was 1 WARNING

GROMACS reminds you: "Do you know what cations don't like? Dog-ions. Do you know what they like? Pie." (Tom Cheatham)

gromacs.qsub: INFO     Setting up queuing system script './local.sh'...
gromacs.cbook: INFO     editing txt = '/home/awsm/mambaforge/envs/mdpow/lib/python3.11/site-packages/gromacs/templates/local.sh' (9 substitutions)
gromacs.cbook: INFO     edited txt = './local.sh'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0950] output mdp = 'md.mdp'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0950] output ndx = '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/main.ndx'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0950] output tpr = 'md.tpr'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0950] output runscripts = ['./local.sh']
gr

Setting the LD random seed to -1564484619

Generated 1689 of the 357435 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'W'

Coupling 1 copies of molecule type 'BENZ'

Velocities were taken from a Maxwell distribution at 300 K

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.297 nm, buffer size 0.197 nm

Set rlist, assuming 4x4 atom pair-list, to 1.214 nm, buffer size 0.114 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 3 Mb of data


Number of degrees of freedom in T-Coupling group System is 4902.00

There were 2 NOTEs

There was 1 WARNING

GROMACS reminds you: "Everybody Wants to Be Naked and Famous" (Tricky)

gromacs.qsub: INFO     Setting up queuing system script './local.sh'...
gromacs.cbook: INFO     editing txt = '/home/awsm/mambaforge/envs/mdpow/lib/python3.11/site-packages/gromacs/templates/local.sh' (9 substitutions)
gromacs.cbook: INFO     edited txt = './local.sh'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/1000] output mdp = 'md.mdp'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/1000] output ndx = '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/main.ndx'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/1000] output tpr = 'md.tpr'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/1000] output runscripts = ['./local.sh']
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/exa

Setting the LD random seed to -270593

Generated 1689 of the 357435 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'W'

Coupling 1 copies of molecule type 'BENZ'

Velocities were taken from a Maxwell distribution at 300 K

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.297 nm, buffer size 0.197 nm

Set rlist, assuming 4x4 atom pair-list, to 1.214 nm, buffer size 0.114 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 3 Mb of data


{'top': '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/top/pp_system.top',
 'ndx': '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/water/solvation/main.ndx',
 'qscript': ['./local.sh'],
 'mainselection': None,
 'deffnm': 'md',
 'includes': ['/home/awsm/MDPOW/mdpow/top'],
 'maxwarn': 1,
 'couple-intramol': 'no',
 'couple_lambda0': 'vdw',
 'couple_lambda1': 'none',
 'sc_alpha': 0.5,
 'sc_power': 1,
 'sc_sigma': 0.3,
 'separate-dhdl-file': 'no',
 'ref_t': 300.0,
 'gen_temp': 300.0,
 'free_energy': 'yes',
 'couple_moltype': 'BENZ',
 'init_lambda_state': 15,
 'calc_lambda_neighbors': -1,
 'vdw-lambdas': array([0.  , 0.05, 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.6 , 0.65, 0.7 , 0.75,
        0.8 , 0.85, 0.9 , 0.95, 1.  ])}

These could be run on a cluster using the `local.sh` scripts, but with Martini they can be done on a local machine in a reasonable amount of time (10-15 minutes on a desktop machine with a GTX 1080). 

In [8]:
for dir_ in gwat.fep_dirs():
    r = gromacs.run.MDrunner(
        dirname=dir_, deffnm="md", c="md.pdb", cpi=True, v=True, dhdl=True
    )
    r.run()  # runs mdrun in the python shell

gromacs.run : INFO     gmx mdrun -deffnm md -c md.pdb -cpi -v -dhdl
                      :-) GROMACS - gmx mdrun, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0000
Command line:
  gmx mdrun -deffnm md -c md.pdb -cpi -v -dhdl

Reading file md.tpr, VERSION 2023.2 (single precision)
Changing nstlist from 20 to 25, rlist from 1.214 to 1.262

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 4 OpenMP threads 

starting mdrun 'Compound BENZ in water'
50000 steps,   1000.0 ps.
step 49900, remaining wall clock time:     0 s          
Writing final coordinates.
step 50000, remaining wall clock time:     0 s          
               Core t (s)   Wall t (s)        (%)
       Time:      1

Finally, we read the `.edr` files from the disk to get the energy reported by Gromacs and use `alchemlyb` to determine the free energy of solvation.

In [9]:
gwat.convert_edr()
gwat.analyze_alchemlyb()

mdpow.fep   : INFO     [FEP/water] Converting EDR -> XVG.bz2
mdpow.fep   : INFO       /home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0000/md.edr --> /home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0000/md.xvg
                      :-) GROMACS - gmx energy, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini
Command line:
  gmx energy -s /home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0000/md.tpr -f /home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0000/md.edr -odh /home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0000/md.xvg

Opened /home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0000/md.edr as single precision energy file
Reading file /home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0000/md.tpr, VERSION 2023.2 (single precision)
Last energy frame read 250 time 1000.000          

GROMACS reminds you: "Fifty years of programming language research, an



Wrote 8 lambda values with 5001 samples as 2008 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0000/md.xvg


Wrote 8 lambda values with 5001 samples as 2008 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0250/md.xvg


Wrote 8 lambda values with 5001 samples as 2008 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0500/md.xvg


Wrote 8 lambda values with 5001 samples as 2008 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0750/md.xvg


Wrote 8 lambda values with 5001 samples as 2008 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/1000/md.xvg


Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0000/md.xvg


                      :-) GROMACS - gmx energy, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini
Command line:
  gmx energy -s /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0050/md.tpr -f /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0050/md.edr -odh /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0050/md.xvg

Opened /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0050/md.edr as single precision energy file
Reading file /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0050/md.tpr, VERSION 2023.2 (single precision)
Last energy frame read 250 time 1000.000          

GROMACS reminds you: "Fifty years of programming language research, and we end up with C++???" (Richard O'Keefe)

mdpow.fep   : INFO       /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0100/md.edr --> /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0100/md.xvg
                      :-) GROMACS - gmx energ



Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0050/md.xvg


Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0100/md.xvg


Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0200/md.xvg


Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0300/md.xvg


                      :-) GROMACS - gmx energy, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini
Command line:
  gmx energy -s /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0400/md.tpr -f /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0400/md.edr -odh /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0400/md.xvg

Opened /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0400/md.edr as single precision energy file
Reading file /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0400/md.tpr, VERSION 2023.2 (single precision)
Last energy frame read 250 time 1000.000          

GROMACS reminds you: "Fifty years of programming language research, and we end up with C++???" (Richard O'Keefe)

mdpow.fep   : INFO       /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0500/md.edr --> /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0500/md.xvg
                      :-) GROMACS - gmx energ

...



Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0700/md.xvg


Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0750/md.xvg


Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0800/md.xvg


Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0850/md.xvg


                      :-) GROMACS - gmx energy, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini
Command line:
  gmx energy -s /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0900/md.tpr -f /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0900/md.edr -odh /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0900/md.xvg

Opened /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0900/md.edr as single precision energy file
Reading file /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0900/md.tpr, VERSION 2023.2 (single precision)
Last energy frame read 250 time 1000.000          

GROMACS reminds you: "Quite frankly, even if the choice of C were to do *nothing* but keep the C++ programmers out, that in itself would be a huge reason to use C." (Linus Torvalds)

mdpow.fep   : INFO       /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0950/md.edr --> /home/awsm/MDPOW/doc/examples/martini/FE



Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0900/md.xvg


Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/0950/md.xvg


Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/water/VDW/1000/md.xvg


mdpow.fep   : INFO     [FEP/water] Compression complete: '/home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0250/md.xvg.bz2'
mdpow.fep   : INFO     [FEP/water] Compressing dgdl file '/home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0500/md.xvg' with bzip2
mdpow.fep   : INFO     [FEP/water] Compression complete: '/home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0500/md.xvg.bz2'
mdpow.fep   : INFO     [FEP/water] Compressing dgdl file '/home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0750/md.xvg' with bzip2
mdpow.fep   : INFO     [FEP/water] Compression complete: '/home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/0750/md.xvg.bz2'
mdpow.fep   : INFO     [FEP/water] Compressing dgdl file '/home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/1000/md.xvg' with bzip2
mdpow.fep   : INFO     [FEP/water] Compression complete: '/home/awsm/MDPOW/doc/examples/martini/FEP/water/Coulomb/1000/md.xvg.bz2'
mdpow.fep   : INFO     [FEP/water] Compressing dgdl file '/hom

6.01644 (nan)

6.74

## Octanol solvation

Now, we repeat the exact same steps as above but using an `OctanolSimulation`.

In [10]:
from mdpow.equil import OctanolSimulation

oct_sim = OctanolSimulation(
    molecule="BENZ",
    forcefield=MARTINI,
    mdp={
        "energy_minimize": str(EM_FILE.absolute()),
        "MD_relaxed": str(EQ_FILE.absolute()),
        "MD_NPT": str(EQ_FILE.absolute()),
        "MD_restrained": str(RUN_FILE.absolute()),
    },
    distance=4.0,
)
oct_sim.topology(str(BENZENE_ITP))
oct_sim.solvate(struct=MARTINI_BENZENE, maxwarn=1)
oct_sim.energy_minimize(maxwarn=1)
oct_sim.MD_relaxed(runtime=1e3, dt=0.01)

mdpow.equil : INFO     [/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/top] Created topology 'system.top' that includes 'martini_v3.0.0_small_molecules_v1.itp'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/solvation] Solvating with water '/home/awsm/MDPOW/doc/examples/martini/octanol.gro'...
                     :-) GROMACS - gmx editconf, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/solvation
Command line:
  gmx editconf -f /home/awsm/MDPOW/doc/examples/martini/benzene.pdb -o boxed.gro -bt dodecahedron -d 4.0


Back Off! I just backed up boxed.gro to ./#boxed.gro.2#

GROMACS reminds you: "I Want to Know Right Now" (Meatloaf)

                     :-) GROMACS - gmx solvate, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini/Equilibrium/octano

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 3 atoms
Volume: 4050 nm^3, corresponds to roughly 1822500 electrons
No velocities found
    system size :  0.210  0.160  0.248 (nm)
    diameter    :  0.270               (nm)
    center      :  2.987  0.606  2.315 (nm)
    box vectors : 15.000 15.000 18.000 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :4050.00               (nm^3)
    shift       :  3.216  5.597  0.609 (nm)
new center      :  6.203  6.203  2.924 (nm)
new box vectors :  8.270  8.270  8.270 (nm)
new box angles  :  60.00  60.00  90.00 (degrees)
new box volume  : 399.98               (nm^3)

         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functiona

gromacs.cbook: INFO     system total charge qtot = 0
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/solvation] After solvation: total charge qtot = 0 = 0
gromacs.cbook: INFO     system total charge qtot = 0
gromacs.setup: INFO     Building the main index file 'main.ndx'...
                     :-) GROMACS - gmx make_ndx, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/solvation
Command line:
  gmx make_ndx -f ionized.tpr -o main.ndx


Reading structure file
Reading file ionized.tpr, VERSION 2023.2 (single precision)
Reading file ionized.tpr, VERSION 2023.2 (single precision)

Back Off! I just backed up main.ndx to ./#main.ndx.4#

GROMACS reminds you: "Where all think alike, no one thinks very much." (Walter Lippmann)

                     :-) GROMACS - gmx make_ndx, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local

Note that major changes are planned in future for trjconv, to improve usability and utility.
Select group for centering
Selected 4: '__main__'
Select group for output
Selected 0: 'System'



NOTE 2 [file system.top, line 28]:
  For energy conservation with LINCS, lincs_iter should be 2 or larger.


Number of degrees of freedom in T-Coupling group rest is 15339.00
The integrator does not provide a ensemble temperature, there is no system ensemble temperature

NOTE 3 [file /tmp/tmprscq1d76.mdp]:
  NVE simulation with an initial temperature of zero: will use a Verlet
  buffer of 10%. Check your energy drift!


There were 3 NOTEs

GROMACS reminds you: "Science progresses best when observations force us to alter our preconceptions." (Vera Rubin)

gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/em] Energy minimization of struct='/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/solvation/solvated.gro', top='/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/top/system.top', mdp='em.mdp' ...
gromacs.cbook: INFO     editing mdp = '/home/awsm/MDPOW/doc/examples/martini/em.mdp': dict_keys(['maxwarn', 'pp', 'include'])
gromacs.cbook:

...

Number of degrees of freedom in T-Coupling group rest is 15339.00
The integrator does not provide a ensemble temperature, there is no system ensemble temperature

Back Off! I just backed up em.tpr to ./#em.tpr.2#

GROMACS reminds you: "Science progresses best when observations force us to alter our preconceptions." (Vera Rubin)

gromacs.run : WARNING  No 'mdrun_d' binary found so trying 'mdrun' instead.
(Note that energy minimization runs better with mdrun_d.)
/home/awsm/mambaforge/envs/mdpow/lib/python3.11/site-packages/gromacs/run.py:423: AutoCorrectionWarning: No 'mdrun_d' binary found so trying 'mdrun' instead.
(Note that energy minimization runs better with mdrun_d.)
  warnings.warn(wmsg, category=AutoCorrectionWarning)
                      :-) GROMACS - gmx mdrun, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/em
Command line:
  gmx mdrun -v -stepout 10 -deffnm em -c em

Setting the LD random seed to -8923428

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'OCO'
Analysing residue names:
There are:  1705      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

This run will generate roughly 1 Mb of data



Back Off! I just backed up em.log to ./#em.log.2#
Reading file em.tpr, VERSION 2023.2 (single precision)
1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 4 OpenMP threads 


Back Off! I just backed up em.trr to ./#em.trr.2#

Back Off! I just backed up em.edr to ./#em.edr.2#

Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+01
   Number of steps    =         1000
Step=    0, Dmax= 1.0e-02 nm, Epot=  1.02801e+07 Fmax= 2.05315e+07, atom= 5083
Step=    1, Dmax= 1.0e-02 nm, Epot=  7.41994e+06 Fmax= 5.74654e+06, atom= 3110
Step=    2, Dmax= 1.2e-02 nm, Epot=  4.40326e+06 Fmax= 2.44532e+06, atom= 4997
Step=    3, Dmax= 1.4e-02 nm, Epot=  2.73795e+06 Fmax= 9.51879e+05, atom= 1685
Step=    4, Dmax= 1.7e-02 nm, Epot=  1.53428e+06 Fmax= 4.51665e+05, atom= 1684
Step=    5, Dmax= 2.1e-02 nm, E

Setting the LD random seed to -1078465571

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'OCO'

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.125 nm, buffer size 0.025 nm

Set rlist, assuming 4x4 atom pair-list, to 1.100 nm, buffer size 0.000 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 3 Mb of data


{'struct': '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/MD_relaxed/md.gro',
 'top': '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/top/system.top',
 'ndx': '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/solvation/main.ndx',
 'qscript': ['./local.sh'],
 'mainselection': None,
 'deffnm': 'md',
 'includes': ['/home/awsm/MDPOW/mdpow/top',
  '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/top']}

In [11]:
r = gromacs.run.MDrunner(
    dirname=oct_sim.dirs["MD_relaxed"],
    deffnm="md",
    c="md.pdb",
    cpi=True,
    v=True,
)
r.run()  # runs mdrun in the python shell

gromacs.run : INFO     gmx mdrun -deffnm md -c md.pdb -cpi -v
                      :-) GROMACS - gmx mdrun, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/MD_relaxed
Command line:
  gmx mdrun -deffnm md -c md.pdb -cpi -v

Reading file md.tpr, VERSION 2023.2 (single precision)
Changing nstlist from 10 to 50, rlist from 1.1 to 1.228

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
Using 1 MPI thread
Using 4 OpenMP threads 

starting mdrun 'Compound BENZ in octanol in water'
100000 steps,   1000.0 ps (continuing from step 100000,   1000.0 ps).

Writing final coordinates.

Back Off! I just backed up md.pdb to ./#md.pdb.2#
step 100000, remaining wall clock time:     0 s          
               Core t (s

0

In [12]:
oct_sim.MD(
    runtime=1e4, qscript=["local.sh"], dt=0.02
)

r = gromacs.run.MDrunner(
    dirname=oct_sim.dirs["MD_NPT"], deffnm="md", c="md.pdb", cpi=True, v=True
)
r.run()  # runs mdrun in the python shell

mdpow.equil : INFO     Found starting structure '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/MD_relaxed/md.pdb' (instead of '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/MD_relaxed/md.gro').
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/MD_NPT] Setting up MD...
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/MD_NPT] input mdp  = '/home/awsm/MDPOW/doc/examples/martini/eq.mdp'
gromacs.cbook: INFO     editing mdp = '/home/awsm/MDPOW/doc/examples/martini/eq.mdp': dict_keys(['nsteps', 'dt', 'pp', 't', 'include'])
gromacs.cbook: WARNING  Not substituted in 'md.mdp': ['pp', 't']
gromacs.setup: WARNING  Unprocessed mdp option are interpreted as options for grompp:
{'pp': 'processed.top', 't': '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/MD_relaxed/md.cpt'}
/home/awsm/mambaforge/envs/mdpow/lib/python3.11/site-packages/gromacs/setup.py:635: UsageWarning: Unprocessed mdp option are int

Setting the LD random seed to -138428440

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'OCO'

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.167 nm, buffer size 0.067 nm

Set rlist, assuming 4x4 atom pair-list, to 1.108 nm, buffer size 0.008 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

Reading Coordinates, Velocities and Box size from old trajectory

Will read whole trajectory

Using frame at t = 1000 ps

Starting time for run is 0 ps

This run will generate roughly 15 Mb of data


Reading file md.tpr, VERSION 2023.2 (single precision)
Changing nstlist from 10 to 25, rlist from 1.108 to 1.223

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the GPU
Using 1 MPI thread
Using 4 OpenMP threads 

starting mdrun 'Compound BENZ in octanol in water'
500000 steps,  10000.0 ps (continuing from step 500000,  10000.0 ps).

Writing final coordinates.

Back Off! I just backed up md.pdb to ./#md.pdb.2#
step 500000, remaining wall clock time:     0 s          
               Core t (s)   Wall t (s)        (%)
       Time:        0.063        0.016      397.6
                 (ns/day)    (hour/ns)
Performance:      109.567        0.219

GROMACS reminds you: "It Just Tastes Better" (Burger King)

gromacs.run : INFO     MDrun completed ok, returncode = 0


0

In [13]:
goct = mdpow.fep.Goct(simulation=oct_sim, runtime=1e+3, mdp=str(RUN_FILE.absolute()))
goct.setup(dt=0.02)

mdpow.fep   : INFO     Found starting structure '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/MD_NPT/md.pdb' (instead of '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/MD_NPT/md.gro').
mdpow.fep   : INFO     Solvation free energy calculation for molecule BENZ in solvent octanol.
mdpow.fep   : INFO     Base directory is '/home/awsm/MDPOW/doc/examples/martini'
mdpow.fep   : INFO     Using setup directories under 'FEP/octanol': {'coulomb': 'FEP/octanol/Coulomb', 'vdw': 'FEP/octanol/VDW'}
mdpow.fep   : INFO     Default checkpoint file is '/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Goct.fep'
mdpow.fep   : INFO     Preparing coulomb for lambda=0
mdpow.fep   : INFO     Setting dhdl file to edr format
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0000] Setting up MD...
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0000] input mdp  = '/home/awsm/MDPOW/doc/examples/martini/run.mdp'
gromacs.u

Setting the LD random seed to 2146696635

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'OCO'

Coupling 1 copies of molecule type 'BENZ'

Velocities were taken from a Maxwell distribution at 300 K

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.292 nm, buffer size 0.192 nm

Set rlist, assuming 4x4 atom pair-list, to 1.179 nm, buffer size 0.079 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 6 Mb of data


Number of degrees of freedom in T-Coupling group System is 15339.00

There were 2 NOTEs

There was 1 WARNING

GROMACS reminds you: "Why would the backup server database get corrupted anyway?" (Stefan Fleischmann -- system administrator, physicist, optimist.)

gromacs.qsub: INFO     Setting up queuing system script './local.sh'...
gromacs.cbook: INFO     editing txt = '/home/awsm/mambaforge/envs/mdpow/lib/python3.11/site-packages/gromacs/templates/local.sh' (9 substitutions)
gromacs.cbook: INFO     edited txt = './local.sh'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0250] output mdp = 'md.mdp'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0250] output ndx = '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/solvation/main.ndx'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0250] output tpr = 'md.tpr'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/o

Setting the LD random seed to 2111659454

Generated 844 of the 356590 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'OCO'

Coupling 1 copies of molecule type 'BENZ'

Velocities were taken from a Maxwell distribution at 300 K

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.292 nm, buffer size 0.192 nm

Set rlist, assuming 4x4 atom pair-list, to 1.179 nm, buffer size 0.079 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 6 Mb of data


Number of degrees of freedom in T-Coupling group System is 15339.00

There were 2 NOTEs

There was 1 WARNING

GROMACS reminds you: "Why would the backup server database get corrupted anyway?" (Stefan Fleischmann -- system administrator, physicist, optimist.)

gromacs.qsub: INFO     Setting up queuing system script './local.sh'...
gromacs.cbook: INFO     editing txt = '/home/awsm/mambaforge/envs/mdpow/lib/python3.11/site-packages/gromacs/templates/local.sh' (9 substitutions)
gromacs.cbook: INFO     edited txt = './local.sh'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0500] output mdp = 'md.mdp'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0500] output ndx = '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/solvation/main.ndx'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0500] output tpr = 'md.tpr'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/o

...

Number of degrees of freedom in T-Coupling group System is 15339.00

There were 2 NOTEs

There was 1 WARNING

GROMACS reminds you: "Push It Real Good" (Salt 'n' Pepa)

gromacs.qsub: INFO     Setting up queuing system script './local.sh'...
gromacs.cbook: INFO     editing txt = '/home/awsm/mambaforge/envs/mdpow/lib/python3.11/site-packages/gromacs/templates/local.sh' (9 substitutions)
gromacs.cbook: INFO     edited txt = './local.sh'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0950] output mdp = 'md.mdp'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0950] output ndx = '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/solvation/main.ndx'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0950] output tpr = 'md.tpr'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0950] output runscripts = ['./local.sh']
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/exampl

Setting the LD random seed to -134791555

Generated 1689 of the 357435 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'OCO'

Coupling 1 copies of molecule type 'BENZ'

Velocities were taken from a Maxwell distribution at 300 K

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.292 nm, buffer size 0.192 nm

Set rlist, assuming 4x4 atom pair-list, to 1.179 nm, buffer size 0.079 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 6 Mb of data


Number of degrees of freedom in T-Coupling group System is 15339.00

There were 2 NOTEs

There was 1 WARNING

GROMACS reminds you: "Push It Real Good" (Salt 'n' Pepa)

gromacs.qsub: INFO     Setting up queuing system script './local.sh'...
gromacs.cbook: INFO     editing txt = '/home/awsm/mambaforge/envs/mdpow/lib/python3.11/site-packages/gromacs/templates/local.sh' (9 substitutions)
gromacs.cbook: INFO     edited txt = './local.sh'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/1000] output mdp = 'md.mdp'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/1000] output ndx = '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/solvation/main.ndx'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/1000] output tpr = 'md.tpr'
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/1000] output runscripts = ['./local.sh']
gromacs.setup: INFO     [/home/awsm/MDPOW/doc/exampl

Setting the LD random seed to -10389

Generated 1689 of the 357435 non-bonded parameter combinations

Excluding 1 bonded neighbours molecule type 'BENZ'

Excluding 1 bonded neighbours molecule type 'OCO'

Coupling 1 copies of molecule type 'BENZ'

Velocities were taken from a Maxwell distribution at 300 K

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K

Calculated rlist for 1x1 atom pair-list as 1.292 nm, buffer size 0.192 nm

Set rlist, assuming 4x4 atom pair-list, to 1.179 nm, buffer size 0.079 nm

Note that mdrun will redetermine rlist based on the actual pair-list setup

This run will generate roughly 6 Mb of data


{'top': '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/top/pp_system.top',
 'ndx': '/home/awsm/MDPOW/doc/examples/martini/Equilibrium/octanol/solvation/main.ndx',
 'qscript': ['./local.sh'],
 'mainselection': None,
 'deffnm': 'md',
 'includes': ['/home/awsm/MDPOW/mdpow/top'],
 'maxwarn': 1,
 'couple-intramol': 'no',
 'couple_lambda0': 'vdw',
 'couple_lambda1': 'none',
 'sc_alpha': 0.5,
 'sc_power': 1,
 'sc_sigma': 0.3,
 'separate-dhdl-file': 'no',
 'ref_t': 300.0,
 'gen_temp': 300.0,
 'free_energy': 'yes',
 'couple_moltype': 'BENZ',
 'init_lambda_state': 15,
 'calc_lambda_neighbors': -1,
 'vdw-lambdas': array([0.  , 0.05, 0.1 , 0.2 , 0.3 , 0.4 , 0.5 , 0.6 , 0.65, 0.7 , 0.75,
        0.8 , 0.85, 0.9 , 0.95, 1.  ])}

These lambda windows take about twice as long as the water calculations on the author's desktop machine.

In [14]:
for dir_ in goct.fep_dirs():
    r = gromacs.run.MDrunner(
        dirname=dir_, deffnm="md", c="md.pdb", cpi=True, v=True, dhdl=True
    )
    r.run()  # runs mdrun in the python shell

gromacs.run : INFO     gmx mdrun -deffnm md -c md.pdb -cpi -v -dhdl
                      :-) GROMACS - gmx mdrun, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0000
Command line:
  gmx mdrun -deffnm md -c md.pdb -cpi -v -dhdl

Reading file md.tpr, VERSION 2023.2 (single precision)
Changing nstlist from 20 to 25, rlist from 1.179 to 1.224

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 4 OpenMP threads 

starting mdrun 'Compound BENZ in octanol in water'
50000 steps,   1000.0 ps.
step 49900, remaining wall clock time:     0 s          
Writing final coordinates.
step 50000, remaining wall clock time:     0 s          
               Core t (s)   Wall t (s)        (%)
      

In [15]:
goct.convert_edr()
goct.analyze_alchemlyb()

mdpow.fep   : INFO     [FEP/octanol] Converting EDR -> XVG.bz2
mdpow.fep   : INFO       /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0000/md.edr --> /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0000/md.xvg
                      :-) GROMACS - gmx energy, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini
Command line:
  gmx energy -s /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0000/md.tpr -f /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0000/md.edr -odh /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0000/md.xvg

Opened /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0000/md.edr as single precision energy file
Reading file /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0000/md.tpr, VERSION 2023.2 (single precision)
Last energy frame read 250 time 1000.000          

GROMACS reminds you: "Ohne Arbeit waer das Leben oede"



Wrote 8 lambda values with 5001 samples as 2008 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0000/md.xvg


Wrote 8 lambda values with 5001 samples as 2008 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0250/md.xvg


Wrote 8 lambda values with 5001 samples as 2008 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0500/md.xvg


Wrote 8 lambda values with 5001 samples as 2008 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/0750/md.xvg


Wrote 8 lambda values with 5001 samples as 2008 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/1000/md.xvg


Last energy frame read 250 time 1000.000          

GROMACS reminds you: "Ohne Arbeit waer das Leben oede" (Wir Sind Helden)

mdpow.fep   : INFO       /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0050/md.edr --> /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0050/md.xvg
                      :-) GROMACS - gmx energy, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini
Command line:
  gmx energy -s /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0050/md.tpr -f /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0050/md.edr -odh /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0050/md.xvg

Opened /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0050/md.edr as single precision energy file
Reading file /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0050/md.tpr, VERSION 2023.2 (single precision)
Last energy frame read 250 time 1000.000          

GROMACS reminds you



Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0000/md.xvg


Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0050/md.xvg


Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0100/md.xvg


Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0200/md.xvg


                      :-) GROMACS - gmx energy, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini
Command line:
  gmx energy -s /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0300/md.tpr -f /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0300/md.edr -odh /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0300/md.xvg

Opened /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0300/md.edr as single precision energy file
Reading file /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0300/md.tpr, VERSION 2023.2 (single precision)
Last energy frame read 250 time 1000.000          

GROMACS reminds you: "Ohne Arbeit waer das Leben oede" (Wir Sind Helden)

mdpow.fep   : INFO       /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0400/md.edr --> /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0400/md.xvg
                      :-) GROMACS - gmx energy, 2023.2 (-:

Executable:

...



Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0800/md.xvg


Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0850/md.xvg


Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0900/md.xvg


Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0950/md.xvg


                      :-) GROMACS - gmx energy, 2023.2 (-:

Executable:   /usr/local/gromacs/bin/gmx
Data prefix:  /usr/local/gromacs
Working dir:  /home/awsm/MDPOW/doc/examples/martini
Command line:
  gmx energy -s /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/1000/md.tpr -f /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/1000/md.edr -odh /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/1000/md.xvg

Opened /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/1000/md.edr as single precision energy file
Reading file /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/1000/md.tpr, VERSION 2023.2 (single precision)
Last energy frame read 250 time 1000.000          

GROMACS reminds you: "Nothing shocks me. I'm a scientist." (Harrison Ford as Indiana Jones)

mdpow.fep   : INFO     Analysis stride is 1.
mdpow.fep   : INFO     Analysis starts from frame 0.
mdpow.fep   : INFO     Analysis stops at frame None.
mdpow.fep   : INFO     [FEP/octanol] Compressing dgdl file '/h



Wrote 19 lambda values with 5001 samples as 4769 dH data blocks to /home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/1000/md.xvg


mdpow.fep   : INFO     [FEP/octanol] Compression complete: '/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/Coulomb/1000/md.xvg.bz2'
mdpow.fep   : INFO     [FEP/octanol] Compressing dgdl file '/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0000/md.xvg' with bzip2
mdpow.fep   : INFO     [FEP/octanol] Compression complete: '/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0000/md.xvg.bz2'
mdpow.fep   : INFO     [FEP/octanol] Compressing dgdl file '/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0050/md.xvg' with bzip2
mdpow.fep   : INFO     [FEP/octanol] Compression complete: '/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0050/md.xvg.bz2'
mdpow.fep   : INFO     [FEP/octanol] Compressing dgdl file '/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0100/md.xvg' with bzip2
mdpow.fep   : INFO     [FEP/octanol] Compression complete: '/home/awsm/MDPOW/doc/examples/martini/FEP/octanol/VDW/0100/md.xvg.bz2'
mdpow.fep   : INFO     [FEP/octanol] Compressing dgdl file

-6.02913 (nan)

-6.70

## Results

Finally, we can compare the calculated free energies of solvation, taking the difference to find the free energy of transfer and then converting to a partition coefficient.

In [16]:
delta_G, p_OW = mdpow.fep.pOW(gwat, goct)
print(f"log P_ow = {p_OW.value} ± {p_OW.error}")

mdpow.fep   : INFO     [BENZ] transfer free energy water --> octanol calculation
mdpow.fep   : INFO     The solvent is water .
mdpow.fep   : INFO     Using already calculated free energy DeltaA
mdpow.fep   : INFO     The solvent is octanol .
mdpow.fep   : INFO     Using already calculated free energy DeltaA
mdpow.fep   : INFO     [BENZ] Values at T = 300 K
mdpow.fep   : INFO     [BENZ] Free energy of transfer water --> octanol: -12.046 (nan) kJ/mol
mdpow.fep   : INFO     [BENZ] log P_ow: 2.097 (nan)


log P_ow = 2.097278019717457 ± nan


The actual value is $\log P_{ow} \approx 2.13$. The calculated value is 2.34.